## Introduction

This notebook is based on the notebooks made by [Pavel Korobov](https://www.kaggle.com/pkorobov), [Oscar Villarreal Escamilla](https://www.kaggle.com/oxzplvifi), [Arc](https://www.kaggle.com/sgreiner) and [CroDoc](https://www.kaggle.com/crodoc).

Please upvote the original notebooks:

[https://www.kaggle.com/code/pkorobov/pixel-travel-map-rotate-more-links](https://www.kaggle.com/code/pkorobov/pixel-travel-map-rotate-more-links)

[https://www.kaggle.com/code/oxzplvifi/pixel-travel-map](https://www.kaggle.com/code/oxzplvifi/pixel-travel-map)

[https://www.kaggle.com/code/sgreiner/pixel-travel-map-removing-duplicates](https://www.kaggle.com/code/sgreiner/pixel-travel-map-removing-duplicates)

[https://www.kaggle.com/code/crodoc/82409-improved-baseline-santa-2022](https://www.kaggle.com/code/crodoc/82409-improved-baseline-santa-2022)

**Improvements comparing to previous notebooks:**

* Moving down, moving up, moving left, moving right, then pick up lowest rate result.


In [ ]:
import numpy as np
import pandas as pd
import functools as fn
import itertools as it
import time


In [ ]:
IMG_FILE = '/kaggle/input/santa-2022/image.csv'
WH = 257
HWH = 128
ORIGIN = [(64,0),(-32,0),(-16,0),(-8,0),(-4,0),(-2,0),(-1,0),(-1,0)] #origin configuration
LOG_SIZE = 1000
#RUN_TIME = 60 * 60 * 8
RUN_TIME = 60 * 60 * 0.5

PRV_FILE = ''
PRV_IDX = 0
PRV_SPATH = ''
PRV_SPOINT = ''
PRV_COUNT = 0
PRV_X = -129
PRV_Y = -129
PRV_RATE = 0
k = 1
ks = 1
kmx = 16
STEP_GAP_LIST = []
while k <= kmx:
    STEP_GAP_LIST.append(k)
    k += ks
print('STEP_GAP_LIST = ' + str(STEP_GAP_LIST))

START_TIME = time.time()

In [ ]:
if PRV_FILE != '':
    df = pd.read_csv(PRV_FILE)
    PRV_SPATH = df['mn_spath'].iloc[PRV_IDX]
    PRV_SPOINT = df['mn_spoint'].iloc[PRV_IDX]
    PRV_COUNT = df['mn_count'].iloc[PRV_IDX]
    PRV_X = df['mn_x'].iloc[PRV_IDX]
    PRV_Y = df['mn_y'].iloc[PRV_IDX]
    PRV_RATE = df['mn_rate'].iloc[PRV_IDX]
    

In [ ]:
# Functions to map between cartesian coordinates and array indexes
def cartesian_to_array(x, y, shape):
    m, n = shape[:2]
    i = (n - 1) // 2 - y
    j = (n - 1) // 2 + x
    if i < 0 or i >= m or j < 0 or j >= n:
        raise ValueError("Coordinates not within given dimensions.")
    return i, j

def array_to_cartesian(i, j, shape):
    m, n = shape[:2]
    if i < 0 or i >= m or j < 0 or j >= n:
        raise ValueError("Coordinates not within given dimensions.")
    y = (n - 1) // 2 - i
    x = j - (n - 1) // 2
    return x, y

# Functions to map an image between array and record formats
def image_to_dict(image):
    image = np.atleast_3d(image)
    kv_image = {}
    for i, j in product(range(len(image)), repeat=2):
        kv_image[array_to_cartesian(i, j, image.shape)] = tuple(image[i, j])
    return kv_image

def image_to_df(image):
    return pd.DataFrame(
        [(x, y, r, g, b) for (x, y), (r, g, b) in image_to_dict(image).items()],
        columns=['x', 'y', 'r', 'g', 'b']
    )

def df_to_image(df):
    side = int(len(df) ** 0.5)  # assumes a square image
    return df.set_index(['x', 'y']).to_numpy().reshape(side, side, -1)

def get_position(config):
    return fn.reduce(lambda p, q: (p[0] + q[0], p[1] + q[1]), config, (0, 0))

def rotate_link(vector, direction):
    x, y = vector
    if direction == 1:  # counter-clockwise
        if y >= x and y > -x:
            x -= 1
        elif y > x and y <= -x:
            y -= 1
        elif y <= x and y < -x:
            x += 1
        else:
            y += 1
    elif direction == -1:  # clockwise
        if y > x and y >= -x:
            x += 1
        elif y >= x and y < -x:
            y += 1
        elif y < x and y <= -x:
            x -= 1
        else:
            y -= 1
    return (x, y)

def rotate(config, i, direction):
    config = config.copy()
    config[i] = rotate_link(config[i], direction)
    return config

def get_square(link_length):
    link = (link_length, 0)
    coords = [link]
    for _ in range(8 * link_length - 1):
        link = rotate_link(link, direction=1)
        coords.append(link)
    return coords

def get_neighbors(config):
    nhbrs = (
        fn.reduce(lambda x, y: rotate(x, *y), enumerate(directions), config)
        for directions in fn.product((-1, 0, 1), repeat=len(config))
    )
    return list(filter(lambda c: c != config, nhbrs))

# Functions to compute the cost function

# Cost of reconfiguring the robotic arm: the square root of the number of links rotated
def reconfiguration_cost(from_config, to_config):
    nlinks = len(from_config)
    diffs = np.abs(np.asarray(from_config) - np.asarray(to_config)).sum(axis=1)
    return np.sqrt(diffs.sum())

# Cost of moving from one color to another: the sum of the absolute change in color components
def color_cost(from_position, to_position, image, color_scale=3.0):
    return np.abs(image[to_position] - image[from_position]).sum() * color_scale

# Total cost of one step: the reconfiguration cost plus the color cost
def step_cost(from_config, to_config, image):
    from_position = cartesian_to_array(*get_position(from_config), image.shape)
    to_position = cartesian_to_array(*get_position(to_config), image.shape)
    return (
        reconfiguration_cost(from_config, to_config) +
        color_cost(from_position, to_position, image)
    )

def get_direction(u, v):
    """Returns the sign of the angle from u to v."""
    direction = np.sign(np.cross(u, v))
    if direction == 0 and np.dot(u, v) < 0:
        direction = 1
    return direction

# We don't use this elsewhere, but you might find it useful."""
def get_angle(u, v):
    """Returns the angle (in degrees) from u to v."""
    return np.degrees(np.math.atan2(
        np.cross(u, v),
        np.dot(u, v),
    ))


def get_path_to_point(config, point):
    """Find a path of configurations to `point` starting at `config`."""
    path = [config]
    # Rotate each link, starting with the largest, until the point can
    # be reached by the remaining links. The last link must reach the
    # point itself.
    for i in range(len(config)):
        link = config[i]
        base = get_position(config[:i])
        relbase = (point[0] - base[0], point[1] - base[1])
        position = get_position(config[:i+1])
        relpos = (point[0] - position[0], point[1] - position[1])
        radius = fn.reduce(lambda r, link: r + max(abs(link[0]), abs(link[1])), config[i+1:], 0)
        # Special case when next-to-last link lands on point.
        if radius == 1 and relpos == (0, 0):
            config = rotate(config, i, 1)
            if get_position(config) == point:  # Thanks @pgeiger
                path.append(config)
                break
            else:
                continue
        while np.max(np.abs(relpos)) > radius:
            direction = get_direction(link, relbase)
            config = rotate(config, i, direction)
            path.append(config)
            link = config[i]
            base = get_position(config[:i])
            relbase = (point[0] - base[0], point[1] - base[1])
            position = get_position(config[:i+1])
            relpos = (point[0] - position[0], point[1] - position[1])
            radius = fn.reduce(lambda r, link: r + max(abs(link[0]), abs(link[1])), config[i+1:], 0)
    assert get_position(path[-1]) == point
    return path

def get_path_to_configuration(from_config, to_config):
    path = [from_config]
    config = from_config.copy()
    while config != to_config:
        for i in range(len(config)):
            config = rotate(config, i, get_direction(config[i], to_config[i]))
        path.append(config)
    assert path[-1] == to_config
    return path

# Compute total cost of path over image
def total_cost(path, image):
    return fn.reduce(
        lambda cost, pair: cost + step_cost(pair[0], pair[1], image),
        zip(path[:-1], path[1:]),
        0,
    )

def config_to_string(config):
    return ';'.join([' '.join(map(str, vector)) for vector in config])

def path_to_sub(path):
    submission = pd.Series(
        [config_to_string(config) for config in path],
        name="configuration",
    )
    return submission

def set_tracker(tracker, p, v):
    ap = cartesian_to_array(p[0], p[1], tracker.shape)
    tracker[ap[0], ap[1]] = v
    
def get_tracker(tracker, p):
    ap = cartesian_to_array(p[0], p[1], tracker.shape)
    return tracker[ap[0], ap[1]]

def string_to_shape(ss):
    points = []
    sa = ss.split("\n")
    for s in sa:
        points.append(string_to_point(s))
    return points

def string_to_path(ss):
    paths = []
    sa = ss.split("\n")
    for s in sa:
        paths.append(string_to_config(s))
    return paths

def string_to_point(ps):
    ps = ps.replace('(', '')
    ps = ps.replace(')', '')
    ap = ps.split(',')
    return (int(ap[0]), int(ap[1]))

def string_to_config(s):
    res = []
    a1 = s.split(';')
    for i in range(len(a1)):
        a2 = a1[i].split(' ')
        res.append((int(a2[0]), int(a2[1])))
    return res

def find_duplicate_points(path, starting_point, ending_point):    
    duplicate_points = {}

    for c in path:
        p = get_position(c)
        if p != starting_point and p != ending_point:
            duplicate_points[p] = duplicate_points.get(p, 0) + 1
            
    return duplicate_points
    
def vector_diff_one(path):    
    for i in range(len(path) - 1):
        for c0, c1 in zip(path[i], path[i+1]):
            if abs(c0[0] - c1[0]) + abs(c0[1] - c1[1]) > 1:
                return False            
    return True

def run_remove(path, starting_point, ending_point, with_log = True):
    if with_log:
        print("-- run remove --")
        print(f"Current length: {len(path)}")
    
    duplicate_points = find_duplicate_points(path, starting_point, ending_point)

    i = len(path) - 2
    while i >= 0 :
        local_p = path[i:i+3]
        p = get_position(local_p[1])
        new_local_p = compress_path(local_p)

        if vector_diff_one(new_local_p) and duplicate_points.get(p, 0) > 1 and len(new_local_p) < 3:
            path = path[:i+1] + path[i+2:]
            duplicate_points[p] -= 1

        i -= 1

    if with_log:
        print(f"New length: {len(path)}")
        
    return path

def is_in_range(p, rg):
    if p[0] >= rg[0] and p[0] <= rg[1] and p[1] >= rg[2] and p[1] <= rg[3]:
        if p[0] >= -HWH and p[0] <= HWH and p[1] >= - HWH and p[1] <= HWH:
            return True
        else:
            return False
    else:
        return False
    
def summarize(path, image, rg):
    cost = total_cost(path, image)

    tracker = np.zeros((WH, WH))
    for c in path:
        p = get_position(c)
        if is_in_range(p, rg):
            set_tracker(tracker, p, 1)
    count = np.sum(tracker)
    
    rate = 0
    if count > 0:
        rate = cost / count
        
    return {'count': count, 'cost': cost, 'rate': rate, 'steps': len(path)}

def compress_path(path):
    
    if len(path) > 2:
        
        new_path = []
        
        max_conf_dist = 1
        
        r = [[] for _ in range(len(path[0]))]
        
        for p in path:
            for i, c in enumerate(p):

                if len(r[i]) == 0 or r[i][-1] != c:
                    
                    if c not in r[i]:
                        r[i].append(c)
                    else:
                        r[i] = r[i][:r[i].index(c) + 1]
                        
                    assert r[i][-1] == c
        
        max_conf_dist = max([len(r_) for r_ in r])
        
        for i in range(max_conf_dist):
            
            new_conf = []
            
            for _, r_ in enumerate(r):
                
                if i < len(r_):
                    c_ = r_[i]
                else:
                    c_ = r_[-1]
                
                new_conf.append(c_)
            
            new_path.append(new_conf)
            
        return new_path
                               
        
    return path


In [ ]:
def solve_left(image, tlp, w, h, sep, scfg = None):
    nx = tlp[0]
    xx = tlp[0] + (w - 1)
    ny = tlp[1] - (h - 1)
    xy = tlp[1]
    rg = (nx, xx, ny, xy)
    print('==> Range: ' + str(rg))
        
    if scfg is not None:
        sep = get_position(scfg)
        start_cfg = scfg
    else:
        path = get_path_to_point(ORIGIN, sep)
        start_cfg = path[-1]

    if not is_in_range(sep, rg):
        return None
        
    tracker = np.zeros((WH, WH))
    
    results = [start_cfg]
    sc = start_cfg
    
    prv_count = 50
    if PRV_SPATH != '':
        results = string_to_path(PRV_SPATH)
        for c in results:
            p = get_position(c)
            if is_in_range(p, rg):
                set_tracker(tracker, p, 1)
        prv_count = np.sum(tracker) + 50
        sc = results[-1]
        
    mn_rate = PRV_RATE
    mn_px = (PRV_X, PRV_Y)
    mn_spath = PRV_SPATH
    mn_spoint = PRV_SPOINT
    mn_count = PRV_COUNT
    mn_rows = []
    while np.sum(tracker) < w * h:
        sp = get_position(sc)        
        for STEP_GAP in STEP_GAP_LIST:
            if np.sum(tracker) == PRV_COUNT + STEP_GAP:
                rdsp = get_position(results[0])
                rdep = get_position(results[-1])
                results = run_remove(results, rdsp, rdep, False)
                results = run_remove(results, rdsp, rdep, False)
                tracker = np.zeros((WH, WH))
                for c in results:
                    p = get_position(c)
                    if is_in_range(p, rg):
                        set_tracker(tracker, p, 1)
                sc = results[-1]
                sp = get_position(sc)        
                sd = summarize(results, image, rg)
                
                mn_rate = sd['rate']
                mn_px = sp
                mn_count = sd['count']

                mn_spoint = ''
                mn_spath = ''
                for c in results:
                    p = get_position(c)
                    if mn_spoint != '':
                        mn_spoint += "\n"
                    mn_spoint += '(' + str(p[0]) + ',' + str(p[1]) + ')'
                    if mn_spath != '':
                        mn_spath += "\n"
                    mn_spath += config_to_string(c)
                mn_rows.append({'mn_x': mn_px[0], 'mn_y': mn_px[1], 'mn_rate': mn_rate, 'mn_count': mn_count, 'mn_spath': mn_spath, 'mn_spoint': mn_spoint})
                    
        if np.sum(tracker) % LOG_SIZE == 0:
            sd = summarize(results, image, rg)
            print('===> ' + str(sd))
            
        # Optimization variables:
        cost = 1e6
        distance = 1e6
        found = False

        # Current configuration:
        sp = get_position(sc)        

        sir = is_in_range(sp, rg)
        if sir:
            set_tracker(tracker, sp, 1)
        
        # Is the location one step on the left of unvisited?
        if not sir:
            next = False
        elif sp[0] == nx: #if we reached the left border
            next = False
        else:
            nxp = (sp[0] - 1, sp[1])
            v = get_tracker(tracker, nxp)
            sir = is_in_range(nxp, rg)
            if not sir:
                next = False
            elif v == 1:
                next = False
            else:
                next = True
                
        # Single-link step:
        for i in range(len(ORIGIN)): #for each arm link
            for d in [-1,1]: #for each direction
                # Rotate link and get new position and vertical displacement:
                c = rotate(sc, i, d)
                p = get_position(c)
                v = get_tracker(tracker, p)
                dx = p[0] - sp[0]
                in_rg = is_in_range(p, rg)

                # Measure cost:
                cost_cur = step_cost(sc, c, image)

                # Must move left unless impossible:
                if v == 0 and in_rg and cost_cur < cost and (dx < 0 or (dx >= 0 and not next)): 
                    nc = c.copy()
                    cost = cost_cur
                    found = True        
                
        if not next:
            # Double-link step:
            for i in range(len(ORIGIN) - 1):
                for d1 in [-1, 1]:
                    for j in range(i + 1, len(ORIGIN)):
                        for d2 in [-1, 1]:
                            # Rotate two separate links, get position and vertical displacement:
                            c = rotate(sc, i, d1)
                            c = rotate(c, j, d2)
                            
                            p = get_position(c)
                            v = get_tracker(tracker, p)
                            dx = p[0] - sp[0]
                            in_rg = is_in_range(p, rg)

                            # Measure cost:
                            cost_cur = step_cost(sc, c, image)

                            # Must move left unless impossible:
                            if v == 0 and in_rg and cost_cur < cost: 
                                nc = c.copy()
                                cost = cost_cur
                                found = True        
                            
            # Tripple-link step:
            for i in range(len(ORIGIN) - 1):
                for d1 in [-1, 1]:
                    for j in range(i + 1, len(ORIGIN)):
                        for d2 in [-1, 1]:
                            for k in range(j + 1, len(ORIGIN)):
                                for d3 in [-1, 1]:
                                    # Rotate three separate links, get position and vertical displacement:
                                    c = rotate(sc, i, d1)
                                    c = rotate(c, j, d2)
                                    c = rotate(c, k, d3)

                                    p = get_position(c)
                                    v = get_tracker(tracker, p)
                                    dx = p[0] - sp[0]
                                    in_rg = is_in_range(p, rg)

                                    # Measure cost:
                                    cost_cur = step_cost(sc, c, image)

                                    # Must move left unless impossible:
                                    if v == 0 and in_rg and cost_cur < cost: 
                                        nc = c.copy()
                                        cost = cost_cur
                                        found = True        

        # If an unvisited point was found, we are done for this step:
        if found:
            sc = nc.copy()
            sp = get_position(sc)
            if is_in_range(sp, rg):
                set_tracker(tracker, sp, 1)
            results.append(sc)        

        # Otherwise, find the nearest unvisited point and go there ignoring the travel map:
        else:
            point = None
            # Search every single pixel of the travel map for unvisited points:
            for i in range(WH): 
                for j in range(WH):
                    p = array_to_cartesian(i, j, image.shape)
                    v = get_tracker(tracker, p)
                    in_rg = is_in_range(p, rg)
                    
                    if v == 0 and in_rg:
                        # Measure the distance to the current point and choose the nearest one:
                        distance_cur = np.sqrt((sp[0] - p[0])**2 + (sp[1] - p[1])**2)
                        if distance_cur < distance:
                            point = p
                            distance = distance_cur

            if point is None:
                break
            else:
                # Go to the nearest unvisited point:
                path = get_path_to_point(results[-1], point)[1:]

                # Output shortest trajectory:
                for c in path:
                    p = get_position(c)
                    if is_in_range(p, rg):
                        set_tracker(tracker, p, 1)
                    results.append(c)                
            
                sc = results[-1]

    sd = summarize(results, image, rg)
    print('===> ' + str(sd))

    rdsp = get_position(results[0])
    rdep = get_position(results[-1])
    results = run_remove(results, rdsp, rdep)
    results = run_remove(results, rdsp, rdep)

    sd = summarize(results, image, rg)
    print('===> ' + str(sd))

    # Return to origin:
    path = get_path_to_configuration(results[-1], start_cfg)[1:]
        
    results.extend(path)

    rdsp = get_position(results[0])
    rdep = get_position(results[-1])
    results = run_remove(results, rdsp, rdep)
    results = run_remove(results, rdsp, rdep)
    
    sd = summarize(results, image, rg)
    
    spoint = ''
    spath = ''
    for c in results:
        p = get_position(c)
        if spoint != '':
            spoint += "\n"
        spoint += '(' + str(p[0]) + ',' + str(p[1]) + ')'
        if spath != '':
            spath += "\n"
        spath += config_to_string(c)

    print('===> Final left: ' + str(sd))
    
    rows = []
    for mn_rw in mn_rows:
        rows.append({'kind': 'up', 'count': sd['count'], 'rate': sd['rate'], 'cost': sd['cost'], 'spath': spath, 'spoint': spoint, 'mn_rate': mn_rw['mn_rate'], 'mn_x': mn_rw['mn_x'], 'mn_y': mn_rw['mn_y'], 'mn_spath': mn_rw['mn_spath'], 'mn_spoint': mn_rw['mn_spoint'], 'mn_count': mn_rw['mn_count']})

    df = pd.DataFrame(rows)

    return df


In [ ]:
def solve_right(image, tlp, w, h, sep, scfg = None):
    nx = tlp[0]
    xx = tlp[0] + (w - 1)
    ny = tlp[1] - (h - 1)
    xy = tlp[1]
    rg = (nx, xx, ny, xy)
    print('==> Range: ' + str(rg))
        
    if scfg is not None:
        sep = get_position(scfg)
        start_cfg = scfg
    else:
        path = get_path_to_point(ORIGIN, sep)
        start_cfg = path[-1]

    if not is_in_range(sep, rg):
        return None
        
    tracker = np.zeros((WH, WH))
    
    results = [start_cfg]
    sc = start_cfg
    
    prv_count = 50
    if PRV_SPATH != '':
        results = string_to_path(PRV_SPATH)
        for c in results:
            p = get_position(c)
            if is_in_range(p, rg):
                set_tracker(tracker, p, 1)
        prv_count = np.sum(tracker) + 50
        sc = results[-1]
        
    mn_rate = PRV_RATE
    mn_px = (PRV_X, PRV_Y)
    mn_spath = PRV_SPATH
    mn_spoint = PRV_SPOINT
    mn_count = PRV_COUNT
    mn_rows = []
    while np.sum(tracker) < w * h:
        sp = get_position(sc)        
        for STEP_GAP in STEP_GAP_LIST:
            if np.sum(tracker) == PRV_COUNT + STEP_GAP:
                rdsp = get_position(results[0])
                rdep = get_position(results[-1])
                results = run_remove(results, rdsp, rdep, False)
                results = run_remove(results, rdsp, rdep, False)
                tracker = np.zeros((WH, WH))
                for c in results:
                    p = get_position(c)
                    if is_in_range(p, rg):
                        set_tracker(tracker, p, 1)
                sc = results[-1]
                sp = get_position(sc)        
                sd = summarize(results, image, rg)

                mn_rate = sd['rate']
                mn_px = sp
                mn_count = sd['count']

                mn_spoint = ''
                mn_spath = ''
                for c in results:
                    p = get_position(c)
                    if mn_spoint != '':
                        mn_spoint += "\n"
                    mn_spoint += '(' + str(p[0]) + ',' + str(p[1]) + ')'
                    if mn_spath != '':
                        mn_spath += "\n"
                    mn_spath += config_to_string(c)
                mn_rows.append({'mn_x': mn_px[0], 'mn_y': mn_px[1], 'mn_rate': mn_rate, 'mn_count': mn_count, 'mn_spath': mn_spath, 'mn_spoint': mn_spoint})
                
        if np.sum(tracker) % LOG_SIZE == 0:
            sd = summarize(results, image, rg)
            print('===> ' + str(sd))
            
        # Optimization variables:
        cost = 1e6
        distance = 1e6
        found = False

        # Current configuration:
        sp = get_position(sc)        

        sir = is_in_range(sp, rg)
        if sir:
            set_tracker(tracker, sp, 1)
        
        # Is the location one step on the left of unvisited?
        if not sir:
            next = False
        elif sp[0] == xx: #if we reached the left border
            next = False
        else:
            nxp = (sp[0] + 1, sp[1])
            v = get_tracker(tracker, nxp)
            sir = is_in_range(nxp, rg)
            if not sir:
                next = False
            elif v == 1:
                next = False
            else:
                next = True
                
        # Single-link step:
        for i in range(len(ORIGIN)): #for each arm link
            for d in [-1,1]: #for each direction
                # Rotate link and get new position and vertical displacement:
                c = rotate(sc, i, d)
                p = get_position(c)
                v = get_tracker(tracker, p)
                dx = p[0] - sp[0]
                in_rg = is_in_range(p, rg)

                # Measure cost:
                cost_cur = step_cost(sc, c, image)

                # Must move left unless impossible:
                if v == 0 and in_rg and cost_cur < cost and (dx > 0 or (dx <= 0 and not next)): 
                    nc = c.copy()
                    cost = cost_cur
                    found = True        
                
        if not next:
            # Double-link step:
            for i in range(len(ORIGIN) - 1):
                for d1 in [-1, 1]:
                    for j in range(i + 1, len(ORIGIN)):
                        for d2 in [-1, 1]:
                            # Rotate two separate links, get position and vertical displacement:
                            c = rotate(sc, i, d1)
                            c = rotate(c, j, d2)
                            
                            p = get_position(c)
                            v = get_tracker(tracker, p)
                            dx = p[0] - sp[0]
                            in_rg = is_in_range(p, rg)

                            # Measure cost:
                            cost_cur = step_cost(sc, c, image)

                            # Must move left unless impossible:
                            if v == 0 and in_rg and cost_cur < cost: 
                                nc = c.copy()
                                cost = cost_cur
                                found = True        
                            
            # Tripple-link step:
            for i in range(len(ORIGIN) - 1):
                for d1 in [-1, 1]:
                    for j in range(i + 1, len(ORIGIN)):
                        for d2 in [-1, 1]:
                            for k in range(j + 1, len(ORIGIN)):
                                for d3 in [-1, 1]:
                                    # Rotate three separate links, get position and vertical displacement:
                                    c = rotate(sc, i, d1)
                                    c = rotate(c, j, d2)
                                    c = rotate(c, k, d3)

                                    p = get_position(c)
                                    v = get_tracker(tracker, p)
                                    dx = p[0] - sp[0]
                                    in_rg = is_in_range(p, rg)

                                    # Measure cost:
                                    cost_cur = step_cost(sc, c, image)

                                    # Must move left unless impossible:
                                    if v == 0 and in_rg and cost_cur < cost: 
                                        nc = c.copy()
                                        cost = cost_cur
                                        found = True        

        # If an unvisited point was found, we are done for this step:
        if found:
            sc = nc.copy()
            sp = get_position(sc)
            if is_in_range(sp, rg):
                set_tracker(tracker, sp, 1)
            results.append(sc)        

        # Otherwise, find the nearest unvisited point and go there ignoring the travel map:
        else:
            point = None
            # Search every single pixel of the travel map for unvisited points:
            for i in range(WH): 
                for j in range(WH):
                    p = array_to_cartesian(i, j, image.shape)
                    v = get_tracker(tracker, p)
                    in_rg = is_in_range(p, rg)
                    
                    if v == 0 and in_rg:
                        # Measure the distance to the current point and choose the nearest one:
                        distance_cur = np.sqrt((sp[0] - p[0])**2 + (sp[1] - p[1])**2)
                        if distance_cur < distance:
                            point = p
                            distance = distance_cur

            if point is None:
                break
            else:
                # Go to the nearest unvisited point:
                path = get_path_to_point(results[-1], point)[1:]

                # Output shortest trajectory:
                for c in path:
                    p = get_position(c)
                    if is_in_range(p, rg):
                        set_tracker(tracker, p, 1)
                    results.append(c)                
            
                sc = results[-1]

    sd = summarize(results, image, rg)
    print('===> ' + str(sd))

    rdsp = get_position(results[0])
    rdep = get_position(results[-1])
    results = run_remove(results, rdsp, rdep)
    results = run_remove(results, rdsp, rdep)

    sd = summarize(results, image, rg)
    print('===> ' + str(sd))
    
    # Return to origin:
    path = get_path_to_configuration(results[-1], start_cfg)[1:]
        
    results.extend(path)

    rdsp = get_position(results[0])
    rdep = get_position(results[-1])
    results = run_remove(results, rdsp, rdep)
    results = run_remove(results, rdsp, rdep)
    
    sd = summarize(results, image, rg)
    
    spoint = ''
    spath = ''
    for c in results:
        p = get_position(c)
        if spoint != '':
            spoint += "\n"
        spoint += '(' + str(p[0]) + ',' + str(p[1]) + ')'
        if spath != '':
            spath += "\n"
        spath += config_to_string(c)

    print('===> Final right: ' + str(sd))
    
    rows = []
    for mn_rw in mn_rows:
        rows.append({'kind': 'up', 'count': sd['count'], 'rate': sd['rate'], 'cost': sd['cost'], 'spath': spath, 'spoint': spoint, 'mn_rate': mn_rw['mn_rate'], 'mn_x': mn_rw['mn_x'], 'mn_y': mn_rw['mn_y'], 'mn_spath': mn_rw['mn_spath'], 'mn_spoint': mn_rw['mn_spoint'], 'mn_count': mn_rw['mn_count']})

    df = pd.DataFrame(rows)

    return df


In [ ]:
def solve_down(image, tlp, w, h, sep, scfg = None):
    nx = tlp[0]
    xx = tlp[0] + (w - 1)
    ny = tlp[1] - (h - 1)
    xy = tlp[1]
    rg = (nx, xx, ny, xy)
    print('==> Range: ' + str(rg))
        
    if scfg is not None:
        sep = get_position(scfg)
        start_cfg = scfg
    else:
        path = get_path_to_point(ORIGIN, sep)
        start_cfg = path[-1]

    if not is_in_range(sep, rg):
        return None
        
    tracker = np.zeros((WH, WH))
    
    results = [start_cfg]
    sc = start_cfg
    
    prv_count = 50
    if PRV_SPATH != '':
        results = string_to_path(PRV_SPATH)
        for c in results:
            p = get_position(c)
            if is_in_range(p, rg):
                set_tracker(tracker, p, 1)
        prv_count = np.sum(tracker) + 50
        sc = results[-1]
        
    mn_rate = PRV_RATE
    mn_px = (PRV_X, PRV_Y)
    mn_spath = PRV_SPATH
    mn_spoint = PRV_SPOINT
    mn_count = PRV_COUNT
    mn_rows = []
    while np.sum(tracker) < w * h:
        sp = get_position(sc)        
        for STEP_GAP in STEP_GAP_LIST:
            if np.sum(tracker) == PRV_COUNT + STEP_GAP:
                rdsp = get_position(results[0])
                rdep = get_position(results[-1])
                results = run_remove(results, rdsp, rdep, False)
                results = run_remove(results, rdsp, rdep, False)
                tracker = np.zeros((WH, WH))
                for c in results:
                    p = get_position(c)
                    if is_in_range(p, rg):
                        set_tracker(tracker, p, 1)
                sc = results[-1]
                sp = get_position(sc)        
                sd = summarize(results, image, rg)

                mn_rate = sd['rate']
                mn_px = sp
                mn_count = sd['count']

                mn_spoint = ''
                mn_spath = ''
                for c in results:
                    p = get_position(c)
                    if mn_spoint != '':
                        mn_spoint += "\n"
                    mn_spoint += '(' + str(p[0]) + ',' + str(p[1]) + ')'
                    if mn_spath != '':
                        mn_spath += "\n"
                    mn_spath += config_to_string(c)
                mn_rows.append({'mn_x': mn_px[0], 'mn_y': mn_px[1], 'mn_rate': mn_rate, 'mn_count': mn_count, 'mn_spath': mn_spath, 'mn_spoint': mn_spoint})
                
        if np.sum(tracker) % LOG_SIZE == 0:
            sd = summarize(results, image, rg)
            print('===> ' + str(sd))
            
        # Optimization variables:
        cost = 1e6
        distance = 1e6
        found = False

        # Current configuration:
        sp = get_position(sc)        

        sir = is_in_range(sp, rg)
        if sir:
            set_tracker(tracker, sp, 1)
        
        # Is the location one step on the left of unvisited?
        if not sir:
            next = False
        elif sp[1] == ny: #if we reached the left border
            next = False
        else:
            nxp = (sp[0], sp[1] - 1)
            v = get_tracker(tracker, nxp)
            sir = is_in_range(nxp, rg)
            if not sir:
                next = False
            elif v == 1:
                next = False
            else:
                next = True
                
        # Single-link step:
        for i in range(len(ORIGIN)): #for each arm link
            for d in [-1,1]: #for each direction
                # Rotate link and get new position and vertical displacement:
                c = rotate(sc, i, d)
                p = get_position(c)
                v = get_tracker(tracker, p)
                dy = p[1] - sp[1]
                in_rg = is_in_range(p, rg)

                # Measure cost:
                cost_cur = step_cost(sc, c, image)

                # Must move left unless impossible:
                if v == 0 and in_rg and cost_cur < cost and (dy < 0 or (dy >= 0 and not next)): 
                    nc = c.copy()
                    cost = cost_cur
                    found = True        
                
        if not next:
            # Double-link step:
            for i in range(len(ORIGIN) - 1):
                for d1 in [-1, 1]:
                    for j in range(i + 1, len(ORIGIN)):
                        for d2 in [-1, 1]:
                            # Rotate two separate links, get position and vertical displacement:
                            c = rotate(sc, i, d1)
                            c = rotate(c, j, d2)
                            
                            p = get_position(c)
                            v = get_tracker(tracker, p)
                            dy = p[1] - sp[1]
                            in_rg = is_in_range(p, rg)

                            # Measure cost:
                            cost_cur = step_cost(sc, c, image)

                            # Must move left unless impossible:
                            if v == 0 and in_rg and cost_cur < cost: 
                                nc = c.copy()
                                cost = cost_cur
                                found = True        
                            
            # Tripple-link step:
            for i in range(len(ORIGIN) - 1):
                for d1 in [-1, 1]:
                    for j in range(i + 1, len(ORIGIN)):
                        for d2 in [-1, 1]:
                            for k in range(j + 1, len(ORIGIN)):
                                for d3 in [-1, 1]:
                                    # Rotate three separate links, get position and vertical displacement:
                                    c = rotate(sc, i, d1)
                                    c = rotate(c, j, d2)
                                    c = rotate(c, k, d3)

                                    p = get_position(c)
                                    v = get_tracker(tracker, p)
                                    dy = p[1] - sp[1]
                                    in_rg = is_in_range(p, rg)

                                    # Measure cost:
                                    cost_cur = step_cost(sc, c, image)

                                    # Must move left unless impossible:
                                    if v == 0 and in_rg and cost_cur < cost: 
                                        nc = c.copy()
                                        cost = cost_cur
                                        found = True        

        # If an unvisited point was found, we are done for this step:
        if found:
            sc = nc.copy()
            sp = get_position(sc)
            if is_in_range(sp, rg):
                set_tracker(tracker, sp, 1)
            results.append(sc)        

        # Otherwise, find the nearest unvisited point and go there ignoring the travel map:
        else:
            point = None
            # Search every single pixel of the travel map for unvisited points:
            for i in range(WH): 
                for j in range(WH):
                    p = array_to_cartesian(i, j, image.shape)
                    v = get_tracker(tracker, p)
                    in_rg = is_in_range(p, rg)
                    
                    if v == 0 and in_rg:
                        # Measure the distance to the current point and choose the nearest one:
                        distance_cur = np.sqrt((sp[0] - p[0])**2 + (sp[1] - p[1])**2)
                        if distance_cur < distance:
                            point = p
                            distance = distance_cur

            if point is None:
                break
            else:
                # Go to the nearest unvisited point:
                path = get_path_to_point(results[-1], point)[1:]

                # Output shortest trajectory:
                for c in path:
                    p = get_position(c)
                    if is_in_range(p, rg):
                        set_tracker(tracker, p, 1)
                    results.append(c)                
            
                sc = results[-1]

    sd = summarize(results, image, rg)
    print('===> ' + str(sd))

    rdsp = get_position(results[0])
    rdep = get_position(results[-1])
    results = run_remove(results, rdsp, rdep)
    results = run_remove(results, rdsp, rdep)

    sd = summarize(results, image, rg)
    print('===> ' + str(sd))

    # Return to origin:
    path = get_path_to_configuration(results[-1], start_cfg)[1:]
        
    results.extend(path)

    rdsp = get_position(results[0])
    rdep = get_position(results[-1])
    results = run_remove(results, rdsp, rdep)
    results = run_remove(results, rdsp, rdep)
    
    sd = summarize(results, image, rg)
    
    spoint = ''
    spath = ''
    for c in results:
        p = get_position(c)
        if spoint != '':
            spoint += "\n"
        spoint += '(' + str(p[0]) + ',' + str(p[1]) + ')'
        if spath != '':
            spath += "\n"
        spath += config_to_string(c)

    print('===> Final down: ' + str(sd))
    
    rows = []
    for mn_rw in mn_rows:
        rows.append({'kind': 'up', 'count': sd['count'], 'rate': sd['rate'], 'cost': sd['cost'], 'spath': spath, 'spoint': spoint, 'mn_rate': mn_rw['mn_rate'], 'mn_x': mn_rw['mn_x'], 'mn_y': mn_rw['mn_y'], 'mn_spath': mn_rw['mn_spath'], 'mn_spoint': mn_rw['mn_spoint'], 'mn_count': mn_rw['mn_count']})

    df = pd.DataFrame(rows)

    return df


In [ ]:
def solve_up(image, tlp, w, h, sep, scfg = None):
    nx = tlp[0]
    xx = tlp[0] + (w - 1)
    ny = tlp[1] - (h - 1)
    xy = tlp[1]
    rg = (nx, xx, ny, xy)
    print('==> Range: ' + str(rg))
        
    if scfg is not None:
        sep = get_position(scfg)
        start_cfg = scfg
    else:
        path = get_path_to_point(ORIGIN, sep)
        start_cfg = path[-1]

    if not is_in_range(sep, rg):
        return None
        
    tracker = np.zeros((WH, WH))
    
    results = [start_cfg]
    sc = start_cfg
    
    prv_count = 50
    if PRV_SPATH != '':
        results = string_to_path(PRV_SPATH)
        for c in results:
            p = get_position(c)
            if is_in_range(p, rg):
                set_tracker(tracker, p, 1)
        prv_count = np.sum(tracker) + 50
        sc = results[-1]
    
    mn_rate = PRV_RATE
    mn_px = (PRV_X, PRV_Y)
    mn_spath = PRV_SPATH
    mn_spoint = PRV_SPOINT
    mn_count = PRV_COUNT
    mn_rows = []
    while np.sum(tracker) < w * h:
        sp = get_position(sc)        
        for STEP_GAP in STEP_GAP_LIST:
            if np.sum(tracker) == PRV_COUNT + STEP_GAP:
                rdsp = get_position(results[0])
                rdep = get_position(results[-1])
                results = run_remove(results, rdsp, rdep, False)
                results = run_remove(results, rdsp, rdep, False)
                tracker = np.zeros((WH, WH))
                for c in results:
                    p = get_position(c)
                    if is_in_range(p, rg):
                        set_tracker(tracker, p, 1)
                sc = results[-1]
                sp = get_position(sc)        
                sd = summarize(results, image, rg)

                mn_rate = sd['rate']
                mn_px = sp
                mn_count = sd['count']

                mn_spoint = ''
                mn_spath = ''
                for c in results:
                    p = get_position(c)
                    if mn_spoint != '':
                        mn_spoint += "\n"
                    mn_spoint += '(' + str(p[0]) + ',' + str(p[1]) + ')'
                    if mn_spath != '':
                        mn_spath += "\n"
                    mn_spath += config_to_string(c)
                mn_rows.append({'mn_x': mn_px[0], 'mn_y': mn_px[1], 'mn_rate': mn_rate, 'mn_count': mn_count, 'mn_spath': mn_spath, 'mn_spoint': mn_spoint})
                
        if np.sum(tracker) % LOG_SIZE == 0:
            sd = summarize(results, image, rg)
            print('===> ' + str(sd))
            
        # Optimization variables:
        cost = 1e6
        distance = 1e6
        found = False

        # Current configuration:
        sp = get_position(sc)        

        sir = is_in_range(sp, rg)
        if sir:
            set_tracker(tracker, sp, 1)
        
        # Is the location one step on the left of unvisited?
        if not sir:
            next = False
        elif sp[1] == xy: #if we reached the left border
            next = False
        else:
            nxp = (sp[0], sp[1] + 1)
            v = get_tracker(tracker, nxp)
            sir = is_in_range(nxp, rg)
            if not sir:
                next = False
            elif v == 1:
                next = False
            else:
                next = True
                
        # Single-link step:
        for i in range(len(ORIGIN)): #for each arm link
            for d in [-1,1]: #for each direction
                # Rotate link and get new position and vertical displacement:
                c = rotate(sc, i, d)
                p = get_position(c)
                v = get_tracker(tracker, p)
                dy = p[1] - sp[1]
                in_rg = is_in_range(p, rg)

                # Measure cost:
                cost_cur = step_cost(sc, c, image)

                # Must move left unless impossible:
                if v == 0 and in_rg and cost_cur < cost and (dy > 0 or (dy <= 0 and not next)): 
                    nc = c.copy()
                    cost = cost_cur
                    found = True        
                
        if not next:
            # Double-link step:
            for i in range(len(ORIGIN) - 1):
                for d1 in [-1, 1]:
                    for j in range(i + 1, len(ORIGIN)):
                        for d2 in [-1, 1]:
                            # Rotate two separate links, get position and vertical displacement:
                            c = rotate(sc, i, d1)
                            c = rotate(c, j, d2)
                            
                            p = get_position(c)
                            v = get_tracker(tracker, p)
                            dy = p[1] - sp[1]
                            in_rg = is_in_range(p, rg)

                            # Measure cost:
                            cost_cur = step_cost(sc, c, image)

                            # Must move left unless impossible:
                            if v == 0 and in_rg and cost_cur < cost: 
                                nc = c.copy()
                                cost = cost_cur
                                found = True        
                            
            # Tripple-link step:
            for i in range(len(ORIGIN) - 1):
                for d1 in [-1, 1]:
                    for j in range(i + 1, len(ORIGIN)):
                        for d2 in [-1, 1]:
                            for k in range(j + 1, len(ORIGIN)):
                                for d3 in [-1, 1]:
                                    # Rotate three separate links, get position and vertical displacement:
                                    c = rotate(sc, i, d1)
                                    c = rotate(c, j, d2)
                                    c = rotate(c, k, d3)

                                    p = get_position(c)
                                    v = get_tracker(tracker, p)
                                    dy = p[1] - sp[1]
                                    in_rg = is_in_range(p, rg)

                                    # Measure cost:
                                    cost_cur = step_cost(sc, c, image)

                                    # Must move left unless impossible:
                                    if v == 0 and in_rg and cost_cur < cost: 
                                        nc = c.copy()
                                        cost = cost_cur
                                        found = True        

        # If an unvisited point was found, we are done for this step:
        if found:
            sc = nc.copy()
            sp = get_position(sc)
            if is_in_range(sp, rg):
                set_tracker(tracker, sp, 1)
            results.append(sc)        

        # Otherwise, find the nearest unvisited point and go there ignoring the travel map:
        else:
            point = None
            # Search every single pixel of the travel map for unvisited points:
            for i in range(WH): 
                for j in range(WH):
                    p = array_to_cartesian(i, j, image.shape)
                    v = get_tracker(tracker, p)
                    in_rg = is_in_range(p, rg)
                    
                    if v == 0 and in_rg:
                        # Measure the distance to the current point and choose the nearest one:
                        distance_cur = np.sqrt((sp[0] - p[0])**2 + (sp[1] - p[1])**2)
                        if distance_cur < distance:
                            point = p
                            distance = distance_cur

            if point is None:
                break
            else:
                # Go to the nearest unvisited point:
                path = get_path_to_point(results[-1], point)[1:]

                # Output shortest trajectory:
                for c in path:
                    p = get_position(c)
                    if is_in_range(p, rg):
                        set_tracker(tracker, p, 1)
                    results.append(c)                
            
                sc = results[-1]

    sd = summarize(results, image, rg)
    print('===> ' + str(sd))

    rdsp = get_position(results[0])
    rdep = get_position(results[-1])
    results = run_remove(results, rdsp, rdep)
    results = run_remove(results, rdsp, rdep)

    sd = summarize(results, image, rg)
    print('===> ' + str(sd))

    # Return to origin:
    path = get_path_to_configuration(results[-1], start_cfg)[1:]
        
    results.extend(path)

    rdsp = get_position(results[0])
    rdep = get_position(results[-1])
    results = run_remove(results, rdsp, rdep)
    results = run_remove(results, rdsp, rdep)
    
    sd = summarize(results, image, rg)
    
    spoint = ''
    spath = ''
    for c in results:
        p = get_position(c)
        if spoint != '':
            spoint += "\n"
        spoint += '(' + str(p[0]) + ',' + str(p[1]) + ')'
        if spath != '':
            spath += "\n"
        spath += config_to_string(c)

    print('===> Final up: ' + str(sd))

    rows = []
    for mn_rw in mn_rows:
        rows.append({'kind': 'up', 'count': sd['count'], 'rate': sd['rate'], 'cost': sd['cost'], 'spath': spath, 'spoint': spoint, 'mn_rate': mn_rw['mn_rate'], 'mn_x': mn_rw['mn_x'], 'mn_y': mn_rw['mn_y'], 'mn_spath': mn_rw['mn_spath'], 'mn_spoint': mn_rw['mn_spoint'], 'mn_count': mn_rw['mn_count']})

    df = pd.DataFrame(rows)

    return df


In [ ]:
img_df = pd.read_csv(IMG_FILE)
image = df_to_image(img_df)


In [ ]:
w = 257
h = 257
tlp = (-128, 128)
sep = (0, 0)
scfg = ORIGIN
print('tlp = ' + str(tlp))
print('sep = ' + str(sep))

In [ ]:
while (PRV_SPATH != '' and PRV_COUNT < w * h) or (PRV_SPATH == ''):
    if time.time() - START_TIME > RUN_TIME:
        break

    df_list = []

    df = solve_left(image, tlp, w, h, sep, scfg)
    if df is not None:
        df_list.append(df)

    df = solve_right(image, tlp, w, h, sep, scfg)
    if df is not None:
        df_list.append(df)

    df = solve_up(image, tlp, w, h, sep, scfg)
    if df is not None:
        df_list.append(df)

    df = solve_down(image, tlp, w, h, sep, scfg)
    if df is not None:
        df_list.append(df)

    if len(df_list) > 0:
        sdf = pd.concat(df_list)
        sdf['prv_count'] = PRV_COUNT
        
        sdf = sdf.sort_values(['rate', 'prv_count'], ascending=[True, True])
        sdf.to_csv('results.csv', index=False)

        sdf2 = sdf.sort_values(['rate', 'prv_count', 'kind', 'mn_count', 'mn_rate'], ascending=[True, True, True, True, True])
        sdf2.to_csv('results-mn.csv', index=False)    

        if False:
            df = pd.read_csv('results-mn.csv')
            PRV_SPATH = df['mn_spath'].iloc[PRV_IDX]
            PRV_SPOINT = df['mn_spoint'].iloc[PRV_IDX]
            PRV_COUNT = df['mn_count'].iloc[PRV_IDX]
            PRV_X = df['mn_x'].iloc[PRV_IDX]
            PRV_Y = df['mn_y'].iloc[PRV_IDX]
            PRV_RATE = df['mn_rate'].iloc[PRV_IDX]
        
        break
    else:
        break


In [ ]:
df = sdf2[0:1]
df.to_csv('results-sm.csv', index=False)
df


In [ ]:
spath = df['spath'].iloc[0]
results = spath.split("\n")
sub_df = pd.DataFrame({'configuration': results})
sub_df.to_csv('submission.csv', index=False)
sub_df
